# Answering Business Questions Using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT 
     name,
     type
  FROM sqlite_master
 WHERE type IN ('table','view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. 

    Artist - Genre
*	Regal - Hip-Hop
*	Red Tone - Punk
*	Meteor and the Girls - Pop
*	Slim Jim Bites - Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql
CREATE VIEW tracks_usa AS
SELECT 
      t.track_id,
      g.name genre_name
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
WHERE c.country = 'USA';

(sqlite3.OperationalError) table tracks_usa already exists
[SQL: CREATE VIEW tracks_usa AS
SELECT 
      t.track_id,
      g.name genre_name
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
WHERE c.country = 'USA';]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [4]:
%%sql
SELECT 
     genre_name,
     COUNT(track_id) tracks_sold,
     ROUND(CAST(COUNT(track_id) AS FLOAT) / (
                                             SELECT COUNT(track_id) 
                                             FROM tracks_usa
                                             ), 2) tracks_proportion
FROM 
(
 SELECT 
      t.track_id,
      g.name genre_name
 FROM customer c
 INNER JOIN invoice i ON c.customer_id = i.customer_id
 INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id
 WHERE c.country = 'USA'
) tracks_usa
 GROUP BY 1
 ORDER BY 2 DESC;

Done.


genre_name,tracks_sold,tracks_proportion
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


From the above, we can see:

* Blues, Hip-Hop, Pop genre tracks are sold more in USA so Slim Jim Bites, Regal, Meteor and the Girls artists albums can be purchased for the store.

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql
SELECT 
     e.first_name || ' ' || e.last_name employee_name,
     COUNT(c.customer_id) customers,
     ROUND(SUM(i.total),2) total_sales_amount,
     e.title employee_title,
     e.hire_date
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
WHERE employee_title = 'Sales Support Agent'
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee_name,customers,total_sales_amount,employee_title,hire_date
Jane Peacock,212,1731.51,Sales Support Agent,2017-04-01 00:00:00
Margaret Park,214,1584.0,Sales Support Agent,2017-05-03 00:00:00
Steve Johnson,188,1393.92,Sales Support Agent,2017-10-17 00:00:00


From the above, we can see:

* Jane Peacock has made sales worth 1731 Dollars which is impressive but considering other factors like the hire_date he could have been done better.

* Steve Johnson who has joined 6 months later than Jane Peacock has made sales worth 1393 Dollars which is good when compared to his peer Jane.

Next task is to analyze the sales data for customers from each different country. 

calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [6]:
%%sql
DROP VIEW IF EXISTS total_customers;

CREATE VIEW total_customers AS
SELECT 
     country,
     COUNT(DISTINCT c.customer_id) total_customers,
     ROUND(SUM(unit_price), 2) total_sales,
     ROUND(SUM(unit_price) / COUNT(DISTINCT c.customer_id), 2) avg_sales_per_customer,
     ROUND(SUM(unit_price) / SUM(quantity), 2) avg_order_value
FROM customer c
LEFT JOIN invoice i ON i.customer_id = c.customer_id
LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
GROUP BY 1;

Done.
Done.


[]

In [7]:
%%sql
DROP VIEW IF EXISTS other_group;

CREATE VIEW other_group AS
SELECT * 
FROM (
      SELECT t.*,
      CASE 
          WHEN total_customers = 1 THEN 'Other'
          ELSE country
      END AS other
      FROM total_customers t
      );

Done.
Done.


[]

In [8]:
%%sql
DROP VIEW IF EXISTS other_group_sort;

CREATE VIEW other_group_sort AS
SELECT 
      other country,
      SUM(total_customers) total_customers,
      SUM(total_sales) total_sales,
      ROUND(AVG(avg_sales_per_customer), 2) avg_sales_per_customer,
      ROUND(AVG(avg_order_value), 2) avg_order_value
FROM other_group
GROUP BY other
ORDER BY total_sales DESC;

(sqlite3.OperationalError) table other_group_sort already exists
[SQL: CREATE VIEW other_group_sort AS
SELECT 
      other country,
      SUM(total_customers) total_customers,
      SUM(total_sales) total_sales,
      ROUND(AVG(avg_sales_per_customer), 2) avg_sales_per_customer,
      ROUND(AVG(avg_order_value), 2) avg_order_value
FROM other_group
GROUP BY other
ORDER BY total_sales DESC;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [9]:
%%sql
SELECT 
      country,
      total_customers,
      total_sales,
      avg_sales_per_customer,
      avg_order_value
FROM (
      SELECT 
            *,
            CASE 
                WHEN country = 'Other' THEN 1
                ELSE 0
            END AS sort
       FROM other_group_sort
      )
ORDER BY sort

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,0.99
Canada,8,535.59,66.95,0.99
Brazil,5,427.68,85.54,0.99
France,5,389.07,77.81,0.99
Germany,4,334.62,83.66,0.99
Czech Republic,2,273.24,136.62,0.99
United Kingdom,3,245.52,81.84,0.99
Portugal,2,185.13,92.57,0.99
India,2,183.15,91.58,0.99
Other,15,1094.94,73.0,0.99


From the above, we can see:

* USA contributes to more sales when compared to other countries with an average sales per customer `80.4`.

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

Find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [57]:
%%sql
DROP VIEW IF EXISTS true_album; 

CREATE VIEW true_album AS
SELECT
      il.invoice_id,
      il.track_id,
      t.album_id,
      a.title,
      (
       SELECT tr.track_id FROM track tr
       LEFT JOIN album al ON tr.album_id = al.album_id
       WHERE tr.album_id = t.album_id
       EXCEPT
       SELECT tra.track_id FROM invoice_line inv
       LEFT JOIN track tra ON tra.track_id = inv.track_id
       WHERE inv.invoice_id = il.invoice_id
      ) IS NULL 
      AND
      (
       SELECT tra.track_id FROM invoice_line inv
       LEFT JOIN track tra ON tra.track_id = inv.track_id
       WHERE inv.invoice_id = il.invoice_id
       EXCEPT
       SELECT tr.track_id FROM track tr
       LEFT JOIN album al ON tr.album_id = al.album_id
       WHERE tr.album_id = t.album_id
      ) IS NULL true_album
FROM invoice i 
LEFT JOIN invoice_line il ON i.invoice_id = il.invoice_id
LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN album a ON a.album_id = t.album_id
GROUP BY il.invoice_id;

Done.
Done.


[]

In [56]:
%%sql
SELECT * FROM true_album;

(sqlite3.OperationalError) no such column: true_album
[SQL: SELECT * FROM true_album;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [60]:
%%sql
SELECT 
      CASE 
          WHEN true_album = 1 then 'album'
          ELSE 'single track'
      END AS purchase_type,
      COUNT(invoice_id) invoices_total,
      ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (
                           SELECT COUNT(invoice_id)
                           FROM invoice
                           ), 2) invoices_percentage
FROM true_album
GROUP BY true_album;

Done.


purchase_type,invoices_total,invoices_percentage
single track,500,0.81
album,114,0.19


From the above, we can see:

* Most of the customers bought tracks as single track purchase and not full albums.

* Considering purchasing only the most popular tracks from each album from record companies, instead of purchasing every track from an album may increase the overall revenue of the company.

## Conclusion:

Business Questions based on chinook store have been justifie